# 차기 신도시 아파트 공급을 위한 경기도 외각지역 위치 추천 &연도별 아파트 면적 선호도 추이
## 데이터 분석 및 시각화 과정 세미 프로젝트(M조) : 

## 시설별 가중치 계산

**가공 목적**

- 현 거주지 선택의 주된 이유를 근거로 주요 세가지 시설별 가중치를 계산


**처리 사항**

1. 필요없는 칼럼들을 제거
2. 결측치 확인
3. 필요한 관측값만 추출
4. 계산을 위한 데이터 타입 변환
5. 비율을 기반으로 가중치 계산 



**활용 데이터**
- KOSIS : '현거주지 선택이유 주된 응답 자료'

In [1]:
import numpy as np
import pandas as pd

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

## 1. 데이터 불러오기

In [3]:
weighting = pd.read_csv("Pdata/현거주지_선택이유주된응답(2004년).csv", encoding='cp949')
weighting

#데이터 불러오기

,행정구역(시도)별,"가구주의 성,교육정도,연령,직업별",2004,2004.1,2004.2,2004.3,2004.4,2004.5,2004.6,2004.7,2004.8,2004.9,2004.10
0,행정구역(시도)별,"가구주의 성,교육정도,연령,직업별",가구,경제적 능력에 맞추어,사업상직장,옛날부터 살아옴,교통편리,근린생활 시설편리,자녀교육 본인학업,자연환경이 좋아서,자녀육아 및 양육,경제적 가치가 오를 것 같아서,기타
1,전국,계,100,48.8,16.6,14.8,5.4,1,4.6,4.5,1.7,1.1,1.6
2,전국,동부,100,52.7,15.9,9.9,6.2,1.1,5.3,4.3,1.8,1.2,1.5
3,전국,읍면부,100,29.9,19.9,38,1.9,0.4,1.2,5.3,0.8,0.7,1.9
4,전국,단독주택,100,45.5,15.7,24,4.4,0.5,3,3.3,1.2,0.8,1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,제주,중졸,100,43.1,13.8,29.3,3.7,1.6,2.6,4.5,-,-,1.3
284,제주,고졸,100,38.6,14.6,25.6,4.4,0.8,5.7,5.3,2.9,0.8,1.3
285,제주,대졸이상,100,44,15.2,15,4.8,1.5,10.1,5.3,3.3,0.6,0.2
286,제주,취학자녀 있음,100,42.9,10.5,20.3,6.3,1.8,10.2,5,2.6,0.4,-


## 2. 데이터 살펴보기

In [4]:
weighting.info()
# 결측치 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   행정구역(시도)별           288 non-null    object
 1   가구주의 성,교육정도,연령,직업별  288 non-null    object
 2   2004                288 non-null    object
 3   2004.1              288 non-null    object
 4   2004.2              288 non-null    object
 5   2004.3              288 non-null    object
 6   2004.4              288 non-null    object
 7   2004.5              288 non-null    object
 8   2004.6              288 non-null    object
 9   2004.7              288 non-null    object
 10  2004.8              288 non-null    object
 11  2004.9              288 non-null    object
 12  2004.10             288 non-null    object
dtypes: object(13)
memory usage: 29.4+ KB


## 3. 데이터 프레임 변형

### 1) 컬럼 정리

In [5]:
weighting.columns=["행정구역(시도)별","가구주의 성,교육정도,연령,직업별",0,1,2,3,4,5,6,7,8,9,10]
weighting
# 컬럼 이름 정리

,행정구역(시도)별,"가구주의 성,교육정도,연령,직업별",0,1,2,3,4,5,6,7,8,9,10
0,행정구역(시도)별,"가구주의 성,교육정도,연령,직업별",가구,경제적 능력에 맞추어,사업상직장,옛날부터 살아옴,교통편리,근린생활 시설편리,자녀교육 본인학업,자연환경이 좋아서,자녀육아 및 양육,경제적 가치가 오를 것 같아서,기타
1,전국,계,100,48.8,16.6,14.8,5.4,1,4.6,4.5,1.7,1.1,1.6
2,전국,동부,100,52.7,15.9,9.9,6.2,1.1,5.3,4.3,1.8,1.2,1.5
3,전국,읍면부,100,29.9,19.9,38,1.9,0.4,1.2,5.3,0.8,0.7,1.9
4,전국,단독주택,100,45.5,15.7,24,4.4,0.5,3,3.3,1.2,0.8,1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,제주,중졸,100,43.1,13.8,29.3,3.7,1.6,2.6,4.5,-,-,1.3
284,제주,고졸,100,38.6,14.6,25.6,4.4,0.8,5.7,5.3,2.9,0.8,1.3
285,제주,대졸이상,100,44,15.2,15,4.8,1.5,10.1,5.3,3.3,0.6,0.2
286,제주,취학자녀 있음,100,42.9,10.5,20.3,6.3,1.8,10.2,5,2.6,0.4,-


In [6]:
weighting2 = weighting[["행정구역(시도)별","가구주의 성,교육정도,연령,직업별",4,5,6]]
weighting2
# 컬럼 추출후 다시 컬럼 정의

,행정구역(시도)별,"가구주의 성,교육정도,연령,직업별",4,5,6
0,행정구역(시도)별,"가구주의 성,교육정도,연령,직업별",교통편리,근린생활 시설편리,자녀교육 본인학업
1,전국,계,5.4,1,4.6
2,전국,동부,6.2,1.1,5.3
3,전국,읍면부,1.9,0.4,1.2
4,전국,단독주택,4.4,0.5,3
...,...,...,...,...,...
283,제주,중졸,3.7,1.6,2.6
284,제주,고졸,4.4,0.8,5.7
285,제주,대졸이상,4.8,1.5,10.1
286,제주,취학자녀 있음,6.3,1.8,10.2


### 2) 필요 관측치만 추출

In [7]:
weighting3 = pd.concat([pd.DataFrame(weighting2.loc[0]).T,weighting2[weighting2["행정구역(시도)별"]=="경기"]])
weighting3
# 관측값 경기도만 가지고 오기

,행정구역(시도)별,"가구주의 성,교육정도,연령,직업별",4,5,6
0,행정구역(시도)별,"가구주의 성,교육정도,연령,직업별",교통편리,근린생활 시설편리,자녀교육 본인학업
153,경기,계,5.2,1.1,3.4
154,경기,남자,5.3,1,3.6
155,경기,여자,4.6,1.4,2.6
156,경기,15-29세,6.2,0.5,2.5
157,경기,30-39세,4.1,1.5,3.2
158,경기,40-49세,5.1,0.9,6
159,경기,50-59세,4.7,1.3,2.4
160,경기,60세이상,7.2,0.9,-
161,경기,65세이상,7.5,1,-


In [8]:
weighting3.info()
# index 수 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 167
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   행정구역(시도)별           16 non-null     object
 1   가구주의 성,교육정도,연령,직업별  16 non-null     object
 2   4                   16 non-null     object
 3   5                   16 non-null     object
 4   6                   16 non-null     object
dtypes: object(5)
memory usage: 768.0+ bytes


### 3) 칼럼 이름 정리 와 행 제거

In [9]:
weighting4 = weighting3.set_axis([i for i in range(16)],axis="index")
# index 이름 정리 후 다시 결과 저장
weighting4.columns=["행정구역(시도)별","가구주의 성,교육정도,연령,직업별","교통편리","근린생활시설편리","자녀교육 본인학업"]
# columns 이름 정리
weighting4


,행정구역(시도)별,"가구주의 성,교육정도,연령,직업별",교통편리,근린생활시설편리,자녀교육 본인학업
0,행정구역(시도)별,"가구주의 성,교육정도,연령,직업별",교통편리,근린생활 시설편리,자녀교육 본인학업
1,경기,계,5.2,1.1,3.4
2,경기,남자,5.3,1,3.6
3,경기,여자,4.6,1.4,2.6
4,경기,15-29세,6.2,0.5,2.5
5,경기,30-39세,4.1,1.5,3.2
6,경기,40-49세,5.1,0.9,6
7,경기,50-59세,4.7,1.3,2.4
8,경기,60세이상,7.2,0.9,-
9,경기,65세이상,7.5,1,-


In [10]:
weighting5=weighting4.drop(index=0, axis=0)
weighting5
# 필요 없는 관측치 제거

,행정구역(시도)별,"가구주의 성,교육정도,연령,직업별",교통편리,근린생활시설편리,자녀교육 본인학업
1,경기,계,5.2,1.1,3.4
2,경기,남자,5.3,1,3.6
3,경기,여자,4.6,1.4,2.6
4,경기,15-29세,6.2,0.5,2.5
5,경기,30-39세,4.1,1.5,3.2
6,경기,40-49세,5.1,0.9,6
7,경기,50-59세,4.7,1.3,2.4
8,경기,60세이상,7.2,0.9,-
9,경기,65세이상,7.5,1,-
10,경기,초졸이하,4.3,0.7,0.6


### 4) 가중치 계산을 위한 결측치값 변경 과 데이터 타입 변환

In [11]:
weighting6= weighting5.replace({"자녀교육 본인학업": "-"},{"자녀교육 본인학업":np.nan})
weighting6
#결측치값 NaN값을 교환

,행정구역(시도)별,"가구주의 성,교육정도,연령,직업별",교통편리,근린생활시설편리,자녀교육 본인학업
1,경기,계,5.2,1.1,3.4
2,경기,남자,5.3,1,3.6
3,경기,여자,4.6,1.4,2.6
4,경기,15-29세,6.2,0.5,2.5
5,경기,30-39세,4.1,1.5,3.2
6,경기,40-49세,5.1,0.9,6
7,경기,50-59세,4.7,1.3,2.4
8,경기,60세이상,7.2,0.9,NaN
9,경기,65세이상,7.5,1,NaN
10,경기,초졸이하,4.3,0.7,0.6


In [12]:
weighting7=weighting6.astype({"교통편리":"float"})
weighting7=weighting7.astype({"근린생활시설편리":"float"})
weighting7=weighting7.astype({"자녀교육 본인학업":"float"})
weighting7.info()
weighting7
# 데이터 타입변환

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 1 to 15
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   행정구역(시도)별           15 non-null     object 
 1   가구주의 성,교육정도,연령,직업별  15 non-null     object 
 2   교통편리                15 non-null     float64
 3   근린생활시설편리            15 non-null     float64
 4   자녀교육 본인학업           13 non-null     float64
dtypes: float64(3), object(2)
memory usage: 720.0+ bytes


,행정구역(시도)별,"가구주의 성,교육정도,연령,직업별",교통편리,근린생활시설편리,자녀교육 본인학업
1,경기,계,5.2,1.1,3.4
2,경기,남자,5.3,1.0,3.6
3,경기,여자,4.6,1.4,2.6
4,경기,15-29세,6.2,0.5,2.5
5,경기,30-39세,4.1,1.5,3.2
6,경기,40-49세,5.1,0.9,6.0
7,경기,50-59세,4.7,1.3,2.4
8,경기,60세이상,7.2,0.9,NaN
9,경기,65세이상,7.5,1.0,NaN
10,경기,초졸이하,4.3,0.7,0.6


## 4. 가중치 계산

In [13]:
traffic_weighting=weighting7["교통편리"][1]/(weighting7["교통편리"][1]+weighting7["근린생활시설편리"][1]+weighting7["자녀교육 본인학업"][1])
commercial_weighting=weighting7["근린생활시설편리"][1]/(weighting7["교통편리"][1]+weighting7["근린생활시설편리"][1]+weighting7["자녀교육 본인학업"][1])
education_weighting=weighting7["자녀교육 본인학업"][1]/(weighting7["교통편리"][1]+weighting7["근린생활시설편리"][1]+weighting7["자녀교육 본인학업"][1])
traffic_weighting
commercial_weighting
education_weighting

0.5360824742268041

0.1134020618556701

0.35051546391752575